In [2]:
from rsplib.processing import RSPSource, RSPEngine, StreamReasoner, ASPEngine

# Comunicating with RDF Streams on TripleWave Using RSPSource Client

In [6]:
stream0 = RSPSource("http://aarhustrafficdata182955", 4000);
stream0.sgraph()

{'@context': {'generatedAt': {'@id': 'http://www.w3.org/ns/prov#generatedAtTime',
   '@type': 'http://www.w3.org/2001/XMLSchema#dateTime'},
  'sld': 'http://streamreasoning.org/ontologies/SLD4TripleWave#'},
 '@id': 'tr:sGraph',
 'sld:contains': {'@list': []},
 'sld:runUUID': 'c72d2b9c-bd8d-11e7-8d13-1a00a83a5601',
 'sld:streamLocation': 'ws://aarhustrafficdata182955:4040/primus',
 'sld:streamName': 'AarhusTrafficData182955',
 'sld:tBoxLocation': 'https://rsplab.blob.core.windows.net/static/citybench.owl'}

In [7]:
stream1 = RSPSource("http://aarhustrafficdata158505", 4001);
stream1.sgraph()

{'@context': {'generatedAt': {'@id': 'http://www.w3.org/ns/prov#generatedAtTime',
   '@type': 'http://www.w3.org/2001/XMLSchema#dateTime'},
  'sld': 'http://streamreasoning.org/ontologies/SLD4TripleWave#'},
 '@id': 'tr:sGraph',
 'sld:contains': {'@list': []},
 'sld:runUUID': 'c72d2b9c-bd8d-11e7-8d13-1a00a83a5601',
 'sld:streamLocation': 'ws://aarhustrafficdata158505:4041/primus',
 'sld:streamName': 'AarhusTrafficData158505',
 'sld:tBoxLocation': 'https://rsplab.blob.core.windows.net/static/citybench.owl'}

# Comunicating with RSP Engine on TripleWave Using the RSPEngine Client

In [8]:
csparql = RSPEngine("http://csparql", 8182);
csparql.status()

{"name":"csparql","host":"csparql/","runUUID":"50057217-bd8d-11e7-801d-1a00a83a5601","port":8182,"empty_results":false,"inference":false,"timestam_function":false,"backloop":false,"num_streams":0,"num_datasets":0,"num_queries":0}


{'backloop': False,
 'empty_results': False,
 'host': 'csparql/',
 'inference': False,
 'name': 'csparql',
 'num_datasets': 0,
 'num_queries': 0,
 'num_streams': 0,
 'port': 8182,
 'runUUID': '50057217-bd8d-11e7-801d-1a00a83a5601',
 'timestam_function': False}

### Register A Stream Two Alternative APIs

### By Name, creating an alias that corresponds to the engine name

In [9]:
csparql.register_stream("AarhusTrafficData182955", "http://aarhustrafficdata182955:4000/sgraph")

"Stream AarhusTrafficData182955 succesfully registered with IRI http://aarhustrafficdata182955:4000/sgraph"


'Stream AarhusTrafficData182955 succesfully registered with IRI http://aarhustrafficdata182955:4000/sgraph'

#### Register A Second Stream

In [10]:
csparql.register_stream("AarhusTrafficData158505", "http://aarhustrafficdata158505:4001/sgraph")

"Stream AarhusTrafficData158505 succesfully registered with IRI http://aarhustrafficdata158505:4001/sgraph"


'Stream AarhusTrafficData158505 succesfully registered with IRI http://aarhustrafficdata158505:4001/sgraph'

In [11]:
csparql.streams()

[{"streamURL":"AarhusTrafficData158505","status":"RUNNING"},{"streamURL":"AarhusTrafficData182955","status":"RUNNING"}]


[{'status': 'RUNNING', 'streamURL': 'AarhusTrafficData158505'},
 {'status': 'RUNNING', 'streamURL': 'AarhusTrafficData182955'}]

## Register a Query

In [12]:
with open('q1.csparql.txt', 'r') as csparq_query:
   body = csparq_query.read()
   print(body)

CONSTRUCT {?s ?p ?o }
FROM STREAM <AarhusTrafficData182955> [RANGE 3s STEP 1s] 
FROM STREAM <AarhusTrafficData158505> [RANGE 3s STEP 1s] 
WHERE 
    {
        ?s ?p ?o
        
}


In [13]:
csparql.register_query("unionstrm", "STREAM", body) 

{'queryBody': 'REGISTER STREAM unionstrm AS CONSTRUCT {?s ?p ?o }\nFROM STREAM <AarhusTrafficData182955> [RANGE 3s STEP 1s] \nFROM STREAM <AarhusTrafficData158505> [RANGE 3s STEP 1s] \nWHERE \n    {\n        ?s ?p ?o\n        \n}'}
"Query unionstrm succesfully registered"


'Query unionstrm succesfully registered'

## Register Observer

In [14]:
csparql.register_observer("unionstrm", "default", {"host":"csparql","type":"ws","port":9102,"name":"default"})

"http://csparql:8182/queries/unionstrm/observers/default"


'http://csparql:8182/queries/unionstrm/observers/default'

In [15]:
csparql.register_stream("UnionStreamOut", "http://csparql:8182/queries/unionstrm/observers/default")

"Stream UnionStreamOut succesfully registered with IRI http://csparql:8182/queries/unionstrm/observers/default"


'Stream UnionStreamOut succesfully registered with IRI http://csparql:8182/queries/unionstrm/observers/default'

In [16]:
with open('q2.csparql.txt', 'r') as csparq_query:
   body2 = csparq_query.read()
   print(body2)

CONSTRUCT {?s ?p ?o} 
FROM STREAM <UnionStreamOut> [RANGE 30s STEP 5s]
WHERE { 
     ?s ?p ?o 
}


In [17]:
csparql.register_query("unionstrmNone", "STREAM", body2)

{'queryBody': 'REGISTER STREAM unionstrmNone AS CONSTRUCT {?s ?p ?o} \nFROM STREAM <UnionStreamOut> [RANGE 30s STEP 5s]\nWHERE { \n     ?s ?p ?o \n}'}
"Query unionstrmNone succesfully registered"


'Query unionstrmNone succesfully registered'

In [18]:
csparql.register_observer("unionstrmNone", "default", {"host":"csparql","type":"ws","port":9103,"name":"default"})

"http://csparql:8182/queries/unionstrmNone/observers/default"


'http://csparql:8182/queries/unionstrmNone/observers/default'

In [28]:
stream0.start() 

{'msg': 'streamCreated'}

In [29]:
stream1.start()

{'msg': 'streamCreated'}